In [2]:
#Enviroment class

import sys
from contextlib import closing
from io import StringIO
from gym import utils
from gym.envs.toy_text import discrete
import numpy as np
 
 
from random import seed
from random import randint
 


# maze construction:
# '|' and '-'are walls, '+'are junctions
# ':' are spaces not available
# 'H' is home and 'W' is the place with 'water'
#  'Food' is in the house.
# The agent needs to press a button to access water and food
# The agent jumps every two places when moving horizontally or vertically.


MAP = [                                                                                                                                                                                       
    "+---------------------------------+",
    "|1:3:5:7:9:1:3:5:7:9:1:3:5:7:9:1:3|",    
    "| +-----+ +-----+ +-----+ +-----+ |",    
    "| | : : | | : : | | : : | | : : | |",#3= 3,5,7,11,13,15,19,21,23,27,29,31 -
    "| +-+ +-+-+-+ +-+ +-+ +-+-+-+ +-+ |",    
    "|   | : : : : |     | : : : : |   |",#5= 5,7,9,11,13,21,23,25,27,29 -
    "| +-+ +-+ +-+ +-+ +-+ +-+ +-+ +-+ |",    
    "| | : : | | : : | | : : | | : : | |",#7= 3,5,7,9,11,13,15,19,21,23,25,27,29,31 +  
    "| +-----+ +-----+-+-----+ +-----+ |",    
    "|       | : : : : : : : : |       |",#9= 9,11,13,15,17,19,21,23,25 -   
    "| +-----+ +-----+ +-----+ +-----+ |",    
    "| | : : | | : : | | : : | | : : | |",#11= 3,5,7,9,11,13,15,17,19,21,23,25,27,29,31 +   
    "| +-+ +-+ +-+ +-+ +-+ +-+ +-+ +-+ |",    
    "|   | : : : : | | | | : : : : |   |",#13= 5,7,9,11,13,17,21,23,25,27,29 +  
    "| +-+ +-+-+-+ +-+ +-+ +-+-+-+ +-+ |",    
    "| | : : | | : : | | : : | | : : | |",#15= 3,5,7,11,13,15,17,19,21,23,27,29,31 +    
    "|-+-----+-+-----+ +-----+ +-----+ |",    
    "|H: : : : : : : : |               |",#17= 1,3,5,7,9,11,13,15,17 -     
    "|-+-----+-+-----+ +-----+ +-----+ |",    
    "| | : : | | : : | | : : | | : :W| |",#19= 3,5,7,11,13,15,17,19,21,23,27,29,31 + 
    "| +-+ +-+-+-+ +-+ +-+ +-+-+-+ +-+ |",    
    "|   | : : : : | | | | : : : : |   |",#21= 5,7,9,11,13,17,21,23,25,27,29 +     
    "| +-+ +-+ +-+ +-+ +-+ +-+ +-+ +-+ |",    
    "| | : : | | : : | | : : | | : : | |",#23= 3,5,7,9,11,13,15,17,19,21,23,25,27,29,31 +
    "| +-----+ +-----+ +-----+ +-----+ |",    
    "|       | : : : : : : : : |       |",#25= 9,11,13,15,17,19,21,23,25 -   
    "| +-----+ +-----+-+-----+ +-----+ |",    
    "| | : : | | : : | | : : | | : : | |",#27= 3,5,7,9,11,13,15,19,21,23,25,27,29,31 +    
    "| +-+ +-+ +-+ +-+ +-+ +-+ +-+ +-+ |",    
    "|   | : : : : |     | : : : : |   |",#29= 5,7,9,11,13,21,23,25,27,29 -    
    "| +-+ +-+-+-+ +-+ +-+ +-+-+-+ +-+ |",    
    "| | : : | | : : | | : : | | : : | |",#31= 3,5,7,11,13,15,19,21,23,27,29,31 -  
    "| +-----+ +-----+ +-----+ +-----+ |",    
    "| : : : : : : : : : : : : : : : : |",
    "+---------------------------------+",
 ]

P = {}

class Maze(discrete.DiscreteEnv):
    """
   
    Destinations:
    - W(ater)
    - H(ome)
 
    Actions:
    There are 5 discrete deterministic actions:
    - 0: move down
    - 1: move up
    - 2: move right
    - 3: move left
    - 4: drink water (press button)
    
 
    Rewards:
    -100 for crash a wall
    -10,000 for going outside the maze  
    -100 for being outside the maze
    -100 for pressing button outside water place
    -1 default per-step
    -exponential function for times in home, without water or without having 'food'
    -1 for being in known place and +100 for being in a new place
    +quadratic function for times between drinking or eating 

 
    Rendering:
    - blue: mouse
    - magenta: destination
 
    state space is represented by:
        (mouse_row, mouse_col, time_in_Maze, time_in_WOWater, time_in_Home, un_explored_Place)
    """
 
    metadata = {"render.modes": ["human", "ansi"]}
    buttonpressed = False
    dest=[19,31]  #water place
    home=[17,1]   #Home
    desc = np.asarray(MAP, dtype="c")

    locs = locs = [(0, 0), (19, 31)]

    num_states = 2178000
    num_rows = 33
    num_columns = 33

    timeMaze =10
    timeWOWater=10
    timeHome=10
    unExplored=2

    max_row = num_rows - 2
    max_col = num_columns - 2

    initial_state_distrib = np.zeros(num_states+1)
    num_actions = 5

    done=False

    taxi_loc=[0,0]


    def __init__(self):

        self.desc = np.asarray(MAP, dtype="c")
 
        self.locs = locs = [(0, 0), (19, 31)]
 
        num_states = 2178000
        num_rows = 33
        num_columns = 33
        timeMaze =10
        timeWOWater=10
        timeHome=10
        unExplored=2
        home=[17,1]
        max_row = num_rows - 2
        max_col = num_columns - 2
        self.initial_state_distrib = np.zeros(num_states+1)
        num_actions = 5
        
        self.P = {
            state: {action: [] for action in range(self.num_actions)}
            for state in range(self.num_states)
        }
      
        done=False
        taxi_loc=[0,0]


    def inicial(self):    #function to fill P table 
          for row in range(self.num_rows):
              for col in range(self.num_columns):
                for timeM in range(self.timeMaze):
                  for timeWOW in range(self.timeWOWater):
                    for timeH in range(self.timeHome):
                        for unExp in range(self.unExplored):

                          state = self.encode(row, col,timeM,timeWOW,timeH, unExp)
                          self.initial_state_distrib[state] += 1
                    
                          for action in range(self.num_actions):
                        
                              new_row, new_col = row, col
                              reward = -1  # default reward 
                              done = False
                              taxi_loc = [row, col]
                              buttonpressed = False

                              reward = reward - 1*((np.exp(timeM/2)/15)*1000-(np.exp(0/2)/15)*1000)       #reward for time in maze (without food)
                              reward = reward - 1*((np.exp(timeWOW/2)/15)*1000-(np.exp(0/2)/15)*1000)     #reward for time without water
                              reward = reward - 1*((np.exp(timeH/2)/15)*1000-(np.exp(0/2)/15)*1000)       #reward for staying at home

                              if unExp == 0: #reward for exploration (+100 new place or -1 for being in a known place)
                                reward = reward-1 
                              else:
                                reward = reward+100
                           
                              if action == 0 : #down
                                if new_row in range(0,self.num_rows):
                                  if self.desc[row+1,col] != b"-":
                                    new_row = min(row + 2, self.max_row)
                                  elif self.desc[row+1,col]  == b"-":
                                    reward =reward-100                        #reward for crashing a wall
                                    new_row =  row
                                else:
                                  reward =reward-10000                        #reward for going outside the maze
                                  new_row =  row
      
                              elif action == 1 : #up
                                if new_row in range(0,self.num_rows):
                                  if self.desc[row-1,col] != b"-":
                                    new_row = min(row - 2, self.max_row)
                                  elif self.desc[row-1,col]  == b"-":
                                    reward =reward -100                       #reward for crashing a wall
                                    new_row =  row
                                else:
                                  reward =reward-10000                        #reward for going outside the maze
                                  new_row =  row
      
                              if action == 2: #right
                                if col in range(0,self.num_columns):
                                  if self.desc[row, col + 1] == b":":
                                    new_col = min(col + 2, self.max_col)
                                  else:
                                    reward = reward-100                       #reward for crashing a wall
                                    new_col = col
                                else:
                                  reward =reward-10000                        #reward for going outside the maze
                                  new_col =  col
      
                              elif action == 3: #left
                                if col in range(0,self.num_columns):
                                  if self.desc[row, col - 1] == b":":
                                    new_col = min(col - 2, self.max_col)
                                  else:
                                    reward = reward-100                       #reward for crashing a wall
                                    new_col = col
                                else:
                                  reward =reward-10000                        #reward for going outside the maze
                                  new_col =  col  
                            
                              elif action == 4:  # press button

                                if taxi_loc == self.dest: #at water place
                                  reward = reward + (timeWOW**2/8-1)*100      #reward for drinking water as a function of time
                                  if timeWOW in range(3,10):

                                    buttonpressed=True
                                    done = True
                                 
                                else:  # presses button that doesn't exist
                                    reward = reward-100                       #reward for pressing a button that doesn't exist
                      

                                if taxi_loc == self.home: #at home
                               
                                    reward = reward + (timeM**2/8-1)*100      #reward for eating at home as a function of time
                                    if timeM in range(3,10):
                                      buttonpressed=True
                                   
                                else:
                                    reward = reward-100                       #reward for pressing a button that doesn't exist
                                    

                              if state<0:
                                print(state, ' ', self.decode(state))
                              new_state = self.encode(
                                  new_row, new_col,timeM,timeWOW,timeH, unExp
                              )
                              if taxi_loc in pos :    
                                reward = reward -100                          #reward for being in a place outside the maze (pos is defined in next cell)
                            
      
                              self.P[state][action].append((1.0, new_state, reward, done))
                                      
          self.initial_state_distrib /= self.initial_state_distrib.sum()
          discrete.DiscreteEnv.__init__(
              self, self.num_states, self.num_actions, self.P, self.initial_state_distrib
          )
 
 
        
    def encode(self, taxi_row, taxi_col,timeMaze,timeWOWater,timeHome, unExplored):
        i = int(taxi_row) 
        i *= 33
        i += int(taxi_col) 
        i *= 10
        i += timeMaze 
        i *= 10
        i += timeWOWater 
        i *= 10
        i += timeHome 
        i *= 2
        i += unExplored
        return int(i)

    def decode(self,i):
        out = []
        out.append(i % 2)
        i = i // 2
        out.append(i % 10)
        i = i // 10
        out.append(i % 10)
        i = i // 10
        out.append(i % 10)
        i = i // 10
        out.append(i % 33)
        i = i // 33
        out.append(i)
        assert 0 <= i < 33
        return out[::-1]

 
    def render(self, mode="human"):
        outfile = StringIO() if mode == "ansi" else sys.stdout
 
        out = self.desc.copy().tolist()
        out = [[c.decode("utf-8") for c in line] for line in out]
        taxi_row, taxi_col, algo, algo1, algo2, algo3= self.decode(self.s)
 
        def ul(x):
            return "_" if x == " " else x
 
        if self.buttonpressed:
            out[taxi_row][taxi_col] = utils.colorize(
                out[taxi_row][ taxi_col], "yellow", highlight=True
            )
            self.buttonpressed=False
 
        else:  # passenger in taxi
            out[ taxi_row][taxi_col] = utils.colorize(
                ul(out[taxi_row][taxi_col ]), "blue", highlight=True
            )
 
        di, dj = self.dest
        out[di][dj ] = utils.colorize(out[ di][dj ], "magenta")
        outfile.write("\n".join(["".join(row) for row in out]) + "\n")
        if self.lastaction is not None:
            outfile.write(
                "  ({})\n".format(
                    ["Down", "Up", "Right", "Left", "Press Button", "Stand still"][
                        self.lastaction
                    ]
                )
            )
        else:
            outfile.write("\n")
 
        # No need to return anything for human
        if mode != "human":
            with closing(outfile):
                return outfile.getvalue()
 
    def step1(self, s, a):
 
        p, s1, r, d = self.P[s][a][0]
        
        self.s = s1
        self.lastaction = a
        
        return p, s1, r, d


  

In [ ]:

#columns allowed in each row allowed
posLoc = [
          [3,5,7,11,13,15,19,21,23,27,29,31],
          [5,7,9,11,13,21,23,25,27,29],
          [3,5,7,9,11,13,15,19,21,23,25,27,29,31],
          [ 9,11,13,15,17,19,21,23,25],
          [3,5,7,9,11,13,15,17,19,21,23,25,27,29,31],
          [5,7,9,11,13,17,21,23,25,27,29],
          [3,5,7,11,13,15,17,19,21,23,27,29,31],

          [1,3,5,7,9,11,13,15,17],

          [3,5,7,11,13,15,17,19,21,23,27,29,31],
          [5,7,9,11,13,17,21,23,25,27,29],
          [3,5,7,9,11,13,15,17,19,21,23,25,27,29,31],
          [9,11,13,15,17,19,21,23,25],
          [3,5,7,9,11,13,15,19,21,23,25,27,29,31],
          [5,7,9,11,13,21,23,25,27,29],
          [3,5,7,11,13,15,19,21,23,27,29,31]
          ]

pos = []

n_pos=[]
for i in range(33):
  for j in range(33):
    n_pos.append([i,j])
n_pos[162]

n_pos2 = [i for i in n_pos if i not in pos]
print(pos)
print(n_pos2)

pos=n_pos2   #pos are locations not allowed


In [3]:
import gym
from sys import exit
from PIL import Image
from time import sleep, time

env = Maze()
env.inicial() #P table construction

In [ ]:
#Q-learning

import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

import random


# Hyperparameters

alpha = 0.01   # learning rate
gamma = 0.6    #temporal discount rate
epsilon = 0.7
#epsilon-greedy strategy. At each step, 
#the agent will decide with probability  1−ϵ  to use the best action for the 
#state it is currently in by looking at the value function, otherwise just make a random choice

row = 17
col = 1
timeM = 0
timeWOW = 0
timeH = 0
unExp = 0
realdone=0

state = env.encode(row, col,timeM,timeWOW,timeH, unExp) #set initial state
count = 0
printea = True  #for printing results at different times

# For plotting metrics
all_epochs = []
all_penalties = []
frames2 = []

for i in range(1, 100001):
    #every episode reset the state to initial 
    epochs, penalties, reward, = 0, 0, 0

    done = False
    ready=False
    row = 17
    col = 1
    timeM = 0
    timeWOW = 0
    timeH = 0
    unExp = 0
    realdone=0

    #times for rewards are counted every XX steps
    epocH=0
    epocW=0
    epocWOW = 0

    loc =[]  #to save the known places
    count = 0
    state = env.encode(row, col,timeM,timeWOW,timeH, unExp) 
    env.s = state

    while not ready:

        row, col,timeM,timeWOW,timeH, unExp = env.decode(state)
        
        
        if random.uniform(0, 1) < epsilon:     #relation between exploring or exploiting
            action = env.action_space.sample() #Explore action space
        else:
            action = np.argmax(q_table[state]) #Exploit learned values
        

        prob, next_state, reward, done = env.step1(state, action)
        new_row, new_col,new_timeM, new_timeWOW,new_timeH, new_unExp = env.decode(next_state)


        if [row,col] == [17,1]:  #counting time at home
          epocH +=1
          if epocH !=0 and epocH%20==0:
            if timeH ==9:  #maximun time
              timeH=9
            else:
              timeH+=1
        else: 
          epocH =0
          timeH =0
          
        
        if [row,col] != [19,31]:  #counting time without water
          epocWOW +=1
          if epocWOW!=0 and epocWOW%20==0:
            if timeWOW == 9:   #maximun time
              timeWOW=9
            else:
              timeWOW+=1
        else: 
          if done:  #reset time only when pressing button
            epocWOW=0
            timeWOW=0
            

        
        if [row,col]!=[17,1]:  #counting time without returning home (food)
          epocM +=1
          if epocM!=0 and epocM%20==0:
            if timeM == 9:   #maximun time
              timeM=9
            else:
              timeM+=1
        else: 
          if timeM > 6 and len(loc)>5:
            loc=[]   #reset known places 
          epocM =0
          timeM=0
          
               
        if [row,col] in loc:   #new place unExp = 1
          unExp =0
        else:
          unExp =1
          loc.append([row,col])


        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward < -100:
          penalties += 10
        if done:
          realdone+=1 

        if printea == True:  #for saving printed frames
          frames2.append({
                'frame': env.render(mode='ansi'),
                'state': state,
                'action': action,
                'reward': reward,
                'timeM':timeM,
                'timeH':timeH,
                'timeWOW':timeWOW,
                'realdone':realdone
                }
            ) 
          

        state = env.encode(new_row, new_col,timeM,timeWOW,timeH, unExp) 
        
        epochs += 1
        
        if realdone== 5:  #after drinking water 5 times, the episode ends
          
          ready=True


    printea =False #only save the frames of some episodes
    if i % 100 == 0:
        printea = True
        print(f"Episode: {i}")
        print(realdone)


In [ ]:
#Performance after Q training

total_epochs, total_penalties = 0, 0
episodes = 100
frames3 = [] 

for _ in range(episodes):

    #set initial state every episode

    epochs, penalties, reward = 0, 0, 0
    done = False
    row = 17
    col = 1
    timeM = 0
    timeWOW = 0
    timeH = 0
    unExp = 0
    ready = False
    epocH=0
    epocW=0
    epocWOW = 0
    realdone = 0
    loc =[]
    count = 0

    state = env.encode(row, col,timeM,timeWOW,timeH, unExp) 
    env.s = state


    while not ready:
        row, col,timeM,timeWOW,timeH, unExp = env.decode(state)
      
        action = np.argmax(q_table[state])
        prob, state, reward, done = env.step1(state, action)
        new_row, new_col,new_timeM,new_timeWOW,new_timeH, new_unExp = env.decode(state)
        
        
        if [row,col] == [17,1]:
          epocH +=1
          if epocH !=0 and epocH%10==0:
            if timeH ==9:
              timeH=9
            else:
              timeH+=1
        else: 
          epocH =0
          timeH =0
          
        
        if [row,col] != [19,31]:
          epocWOW +=1
          if epocWOW!=0 and epocWOW%10==0:
            if timeWOW == 9:
              timeWOW=9
            else:
              timeWOW+=1
        else: 
          if done:
            epocWOW =0
            timeWOW=0

        
        if [row,col]!=[17,1] :
          epocM +=1
          if epocM!=0 and epocM%10==0:
            if timeM == 9:
              timeM=9
            else:
              timeM+=1
        else: 
          if timeM > 7 and len(loc)>5:
            loc=[]
          epocM =0
          timeM=0 
          
          
          
        
        if [row,col] in loc:
          unExp =0
        else:
          unExp =1
          loc.append([row,col])

        
 

        frames3.append({
              'frame': env.render(mode='ansi'),
              'state': state,
              'action': action,
              'reward': reward,
              'timeM':timeM,
              'timeH':timeH,
              'timeWOW':timeWOW,
              'realdone':realdone
              }
          )
        if done:
          realdone+=1
         
        epochs+=1

        if realdone == 5:
          ready = True
        if reward < -100:
            penalties += 10
        state = env.encode(new_row, new_col,timeM,timeWOW,timeH, unExp)
    print(f"Results after {_+1} episodes:")
    print(f"Average timesteps per episode: {total_epochs / (_+1)}")
    print(f"Average penalties per episode: {total_penalties / (_+1)}")

    total_penalties += penalties
    total_epochs += epochs

In [ ]:
#printing frames from the maze

from time import sleep, time
from IPython.display import clear_output
from IPython.display import Image

def print_frames2(frames, t):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}",'\t\t',f"Time in Maze: {frame['timeM']}")
        print(f"State: {frame['state']}",'\t\t',f"Time in Home: {frame['timeH']}")
        print(f"Action: {frame['action']}",'\t\t',f"Time without water: {frame['timeWOW']}")
        print(f"Reward: {int(frame['reward'])}",'\t\t',f"Drink water: {frame['realdone']}")
        sleep(t)


#printing frames of the performance (t is time waiting for the next frame)
print_frames2(frames3, t=0.1)